In [1]:
# Get all required libraries
import pandas as pd
import numpy as np

import os
import random
from scipy import stats
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import backend as K
#import tensorflow_addons as tfa

import transformers

import warnings
warnings.filterwarnings("ignore") 

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.3.0


## Load Data set

In [2]:
# read csv files in the us-patent-phrase-to-phrase-matching
# directory and store them in a list
path = 'us-patent-phrase-to-phrase-matching'
files = os.listdir(path)
# read the csv files
df_train = pd.read_csv(path + '/' + 'train.csv')
df_test = pd.read_csv(path + '/' + 'test.csv')
df_sample = pd.read_csv(path + '/' + 'sample_submission.csv')
parsed = {x: [] for x in ['code', 'title', 'section', 'class', 'subclass', 'group', 'main_group']}
os.chdir(path)
for letter in 'ABCDEFGHY':
    file = f'cpc-section-{letter}_20220201.txt'
    with open(file) as f:
        for line in f:
            vals = line.strip().split('\t')
            if len(vals) == 2:
                parsed['code'].append(vals[0])
                parsed['title'].append(vals[1])
            elif len(vals) == 3:
                parsed['code'].append(vals[0])
                parsed['title'].append(vals[2])
for i in range(len(parsed['code'])):
    code = parsed['code'][i]
    main_group = code.split('/')[-1] if "/" in code else None
    group = code.split('/')[0][4:] if len(code) >= 5 else None
    subclass = code[3] if len(code) >= 4 else None
    class_ = code[1:3] if len(code) >= 3 else None
    section = code[0] if len(code) >= 1 else None
    
    parsed['main_group'].append(main_group)
    parsed['group'].append(group)
    parsed['subclass'].append(subclass)
    parsed['class'].append(class_)
    parsed['section'].append(section)


# merge both dataframes
df_codes = pd.DataFrame.from_dict(parsed)
codes = df_codes.rename(columns = {"code" : "context"})
train_data=pd.merge(df_train,codes[["context","title"]],on="context",how="left")
test_data=pd.merge(df_test,codes[["context","title"]],on="context",how="left")

## Build Model

In [3]:
#TODO - we will enable it after our intial training
# tf.config.optimizer.set_jit(True) 
class Config():
    seed = 42
    epochs = 10
    num_folds = 5
    max_length = 96 # 192
    batch_size = 16 #64
    learning_rate = 2e-5
    weight_decay = 0.01
    base_model = "anferico/bert-for-patents"
    
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)
    
#seed_everything(seed=42)
# strategy = tf.distribute.MirroredStrategy()

,id,anchor,target,context,score,title
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
